In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd
import shutil
import os
import numpy as np

def copy_models_to_kaggle(model_paths):
    os.makedirs('/kaggle/working/models', exist_ok=True)
    new_paths = []
    for model_path in model_paths:
        model_trained_data_size = model_path.split("/")[3]
        model_trained_img_size = model_path.split("/")[5]
        new_model_path = '/kaggle/working/models/' + model_trained_data_size + "_dataset" + "_imgsize_" + model_trained_img_size + "_model.keras"
        new_paths.append(new_model_path)
        if os.path.exists(new_model_path):
            # raise Exception(f"model already exists at {new_model_path}")
            print(f"model already exists at {new_model_path}")
        shutil.copy(model_path, new_model_path)
    return new_paths
 

batch_size = 32
test_dataset_paths = [
    # '/kaggle/input/sample-1800',
    # '/kaggle/input/sample-2500',
    # '/kaggle/input/clean-2500-letters-only',
    # '/kaggle/input/clean-1800-letters-only'
    '/kaggle/input/2500-baa-taa-thaa',
    '/kaggle/input/1800-baa-taa-thaa'
]
models_paths = [
        # '/kaggle/input/1800-model/keras/16/1/model img_size_16.keras',
        # '/kaggle/input/1800-model/keras/32/1/model img_size_32.keras',
        # '/kaggle/input/1800-model/keras/64/1/model img_size_64.keras',
        # '/kaggle/input/1800-model/keras/128/1/model img_size_128.keras',
        # '/kaggle/input/1800-model/keras/256/1/model img_size_256.keras',
        # '/kaggle/input/1800-model/keras/300/1/model img_size_300.keras',

        # '/kaggle/input/2500-model/keras/16/1/model img_size_16.keras',
        # '/kaggle/input/2500-model/keras/32/1/model img_size_32.keras',
        # '/kaggle/input/2500-model/keras/64/1/model img_size_64.keras',
        # '/kaggle/input/2500-model/keras/128/1/model img_size_128.keras',
        # '/kaggle/input/2500-model/keras/256/1/model img_size_256.keras',
        # '/kaggle/input/2500-model/keras/300/1/model img_size_300.keras'
        '/kaggle/input/1800-baa-taa-thaa-model/keras/imgsize_128/1/model_5616_images_img_size_128.keras'
    
]
print("copying models to kaggle to be able to work with WRITE ACESS ...")
copied_models_paths = copy_models_to_kaggle(models_paths)
print(f"loading models in RAM...")
model_path_to_model = {}
for model_path in copied_models_paths:
    print(f"loading model in : {model_path} ...")
    model = load_model(model_path)
    model_path_to_model[model_path] = model

all_datasets_accuracies = {}
for dataset_path in test_dataset_paths:
    dataset_name  = dataset_path.split("/")[-1]
    print(f"Testing dataset : {dataset_path} ...")
    dataset_test_accuracies = {}
    for model_path in copied_models_paths:
        # /kaggle/working/models/2500-model_dataset_imgsize_300_model.keras
        trained_data_size = int(model_path.split("/")[4].split("-")[0])
        print(f"trained data size : {trained_data_size}")
        img_size = int(model_path.split("/")[4].split("_")[-2])
        print(f"image size : {img_size}")
        current_model_name =  str(trained_data_size) + " ," + str(img_size) + "_model"
        print(f"using : {current_model_name} ...")
        model = model_path_to_model[model_path]

        print(f"Testing model : {current_model_name} ...")
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(
            dataset_path,
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)
        print()

        # Predict the output
        steps = int(np.ceil(test_generator.samples / test_generator.batch_size))
        predictions = model.predict(test_generator, steps=steps)
        predictions = predictions.argmax(axis=-1)                    
        true_classes = test_generator.classes
        accuracy = round(accuracy_score(true_classes, predictions), 3)
        dataset_test_accuracies[current_model_name] = accuracy
        print(f"Accuracy of model {current_model_name} on dataset {dataset_name} : {accuracy}")
    all_datasets_accuracies[dataset_name] = dataset_test_accuracies

print(f"all datasets accuracies : {all_datasets_accuracies}")
accuracies_df = pd.DataFrame(all_datasets_accuracies).transpose()
accuracies_df = accuracies_df.style.background_gradient(cmap='tab20_r').set_table_styles([dict(selector="th", props=[("text-align", "center")])])
display(accuracies_df)


2024-04-24 09:02:35.561641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 09:02:35.561747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 09:02:35.691638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


copying models to kaggle to be able to work with WRITE ACESS ...
loading models in RAM...
loading model in : /kaggle/working/models/1800-baa-taa-thaa-model_dataset_imgsize_imgsize_128_model.keras ...
Testing dataset : /kaggle/input/2500-baa-taa-thaa ...
trained data size : 1800
image size : 128
using : 1800 ,128_model ...
Testing model : 1800 ,128_model ...
Found 7443 images belonging to 3 classes.



/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


233/233 ━━━━━━━━━━━━━━━━━━━━ 42s 174ms/step
Accuracy of model 1800 ,128_model on dataset 2500-baa-taa-thaa : 0.333
Testing dataset : /kaggle/input/1800-baa-taa-thaa ...
trained data size : 1800
image size : 128
using : 1800 ,128_model ...
Testing model : 1800 ,128_model ...
Found 5616 images belonging to 3 classes.

  2/176 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step 

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


176/176 ━━━━━━━━━━━━━━━━━━━━ 41s 236ms/step
Accuracy of model 1800 ,128_model on dataset 1800-baa-taa-thaa : 0.952
all datasets accuracies : {'2500-baa-taa-thaa': {'1800 ,128_model': 0.333}, '1800-baa-taa-thaa': {'1800 ,128_model': 0.952}}


,"1800 ,128_model"
2500-baa-taa-thaa,0.333000
1800-baa-taa-thaa,0.952000
